In [3]:
!pip3 install psycopg2


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [17]:
import psycopg2

class Database:
    def __init__(self):
        self.conn = psycopg2.connect(
            host="awesome-hw.sdsc.edu",
            port=5432,
            dbname="nourish",
            user="swhoyle",
            password="dse203#2025"
        )

    def query(self, sql):
        with self.conn.cursor() as cursor:
            cursor.execute(sql)
            columns = [c.name for c in cursor.description]
            rows = cursor.fetchall()
        
        results = [dict(zip(columns, row)) for row in rows]
        return results


db = Database()

In [19]:
import json

entities = ["city", "country", "county", "state"]

for entity in entities:
    results = db.query(f"SELECT * FROM entity_{entity};")
    with open(f"data/{entity}.json", "w") as f:
        json.dump(results, f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/city.json'